### Кластеризация пользователей Twitter на основе текста твитов.

In [1]:
import pandas as pd
import numpy as np

from pandasticsearch import DataFrame
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

Импорт библиотек для обработки текстов.

In [2]:
import os
import regex
import nltk

from nltk.corpus import stopwords
from stop_words import get_stop_words
from pymystem3 import Mystem
from string import punctuation
from emoji import UNICODE_EMOJI

In [3]:
nltk.data.path.append(os.path.abspath(os.getcwd() + '/data'))
nltk.download('stopwords', download_dir='./data/')

stem = Mystem()
russian_stopwords = set(stopwords.words('russian'))
russian_stopwords.update(['че', 'чё', 'мм', 'ммм', 'мммм', 'год',
                          'кто', 'что', 'кого', 'чего', 'самый', 'самая',
                          'аля', 'из', 'за', 'то', 'ой'])
russian_stopwords.update(get_stop_words('russian'))

[nltk_data] Downloading package stopwords to ./data/...
[nltk_data]   Package stopwords is already up-to-date!


Удаляем из текста стоп-слова, знаки препинания, слова с латинскими буквами и ссылки.
Также приводим русские слова в нормальную форму.

In [4]:
def is_symbols(word):
    return bool(regex.match(fr'[{punctuation} \n_`«»“”️–]', word))

def is_emoji(word):
    return word in UNICODE_EMOJI

def is_latin(word):
    return bool(regex.match(r'\p{IsLatin}', word))

def is_number(word):
    return bool(regex.match(r'\d', word))

def is_url(word):
    return bool(regex.match(r'http[s]?://\S+', word))

def clear(sentence):
    return ' '.join([w if w[0] != '@' and not is_url(w)
                     else ''
                     for w in sentence.split(' ') if len(w) > 0])

def preprocess_tweets(tweets):
    result = ''
    for tweet in tweets:
        text = []
        for t in stem.lemmatize(clear(tweet).lower()):
            token = t.strip()
            if not token == '' \
            and not is_number(token) \
            and not is_emoji(token) \
            and not is_symbols(token) \
            and token not in russian_stopwords:
                text.append(token)
        result += ' '.join(text)
    return result

Подключаемся к инстансу Elasticsearch, извлекаем из него данные и загружаем в датафрейм.

In [19]:
df = DataFrame \
    .from_es(url='http://localhost:9200', index='twittify-tweets', compat=7)

Выбираем твиты на русском языке.

In [20]:
df = df.filter(df.language == 'ru') \
    .limit(10_000) \
    .select('nlikes', 'nreplies', 'nretweets', 'tweet', 'username', 'name') \
    .to_pandas()

Игнорируем поля Elasticsearch.

In [21]:
df = df.drop(['_index', '_type', '_id', '_score', '_ignored'], axis=1)

Группируем твиты по пользователям, данные в численных столбцах усредняем, тексты твитов обрабатываем и объединяем.

In [22]:
df = df.groupby(df.username).aggregate(list)

df.nlikes = df.nlikes.apply(np.mean)
df.nreplies = df.nreplies.apply(np.mean)
df.nretweets = df.nretweets.apply(np.mean)
df.name = df.name.apply(lambda s: s[0])

df.tweet = df.tweet.apply(preprocess_tweets)

Весь корпус текстов пропускаем через TF-IDF.

In [24]:
corpus = df.tweet

tfidf = TfidfVectorizer(
    min_df=5,
    max_df=0.95
)
tfidf.fit(corpus)
text = tfidf.transform(corpus)

Кластеризуем алгоритмом K-Means.
Добавляем метки кластера каждому пользователю.

In [25]:
n_clusters = int(df.shape[0] / 40)
clusters = KMeans(
    n_clusters=n_clusters
).fit(text)

df['cluster'] = clusters.labels_

Для поиска наиболее полезных ключевых слов группируем тексты твитов по кластерам.
В итоге получится `k` документов, которые мы снова пропускаем через TF-IDF.

In [52]:
corpus_cluster = df.groupby(df.cluster) \
    .aggregate(list) \
    .tweet \
    .apply(' '.join)

tfidf_cluster = TfidfVectorizer(
    min_df=5,
    max_df=0.95
)
text_cluster = tfidf_cluster.fit_transform(corpus_cluster)

Для каждого кластера выводим `n` наиболее часто встречающихся (ключевых) слов, которые используем для присваивания тэга.

In [53]:
for c, r in pd.DataFrame(text_cluster.todense()).iterrows():
    frequent = [tfidf_cluster.get_feature_names()[i] for i in np.argsort(r)[-10:]]
    print('Cluster #{}: {}'.format(c, ' '.join(frequent)))

Cluster #0: начинать женщина ребенок вообще какой что думать делать понимать то
Cluster #1: выборы задерживать минск обыск то власть путин суд дело навальный
Cluster #2: уборка услуга лс интернет прием мегафон сайт магазин ярмарка заказ
Cluster #3: брать игра какой понимать слово прочитывать вообще выкидывать кукла то
Cluster #4: клуб цска мяч зенит тренер то гол игрок спартак матч
Cluster #5: посылка техника москва почта подмосковье офисный номер отделение ремонт обслуживание
Cluster #6: совет парк состояться посольство иностранный посол проходить камчатка российский федерация
Cluster #7: аниме думать нравиться делать игра какой понимать что вообще то
Cluster #8: уставать помнить либо спрашивать минута лапа вообще то понимать жрать
Cluster #9: мойпервыйтвить продавать монета вложение конец запуск заработок купить зарабатывать биржа
Cluster #10: понимать дерево санкт решать друг делать работать сделать мчс то
Cluster #11: подписка код фильм мода оттенок скачать платформа электронный кн

Предсказываем кластер произвольного пользователя и выводим несколько рекомендуемых пользователей.

In [28]:
user = df.sample(n=1)
tweet = tfidf.transform(user.tweet)
cluster = clusters.predict(tweet)[0]

print(f'User: {user.index[0]}')
print(f'Cluster: {cluster}')

User: evapobeda20
Cluster: 2


In [31]:
recommended = df[df['cluster'] == cluster].sample(n=3)

for i in range(recommended.shape[0]):
    print(f'Full name: {recommended.name.iloc[i]}')
    print(f'Recommended: {recommended.index[i]}')


Full name: Издательство «Чёрная Сотня»
Recommended: chernaya100book
Full name: Пьяный Батя
Recommended: pyanibatya
Full name: xen bezdenezhnykh
Recommended: saltyears_
